In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import sys
pio.renderers.default = 'browser'

In [2]:
#from sqlalchemy import create_engine
import mysql.connector

In [4]:
# Append the parent directory to sys.path
import os
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

## Code below will populate data with correct columns

In [ ]:
from dotenv import load_dotenv
import mysql.connector
import os

from research.trades_db_utils import create_connection

load_dotenv()
DB_HOSTNAME = os.getenv('DB_HOSTNAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

conn = create_connection(DB_HOSTNAME, 'CoinTrades', DB_USER, DB_PASSWORD)

query = """
SELECT datetime, open, high, low, close, volume, signal
FROM crypto_data
ORDER BY datetime
"""
cursor = conn.cursor()

cursor.execute(query)


row = cursor.fetchall()
conn.close()

df = pd.DataFrame(row, columns=['coin_name', 'signature', 'sol_amount', 'token_amount', 'is_buy', 'timestamp'])
df.sort_values(by='timestamp', ascending=False, inplace=True)
df.head(5)

In [5]:
# Step 1: Connect to MySQL database and fetch data (NEW)
'''load_dotenv()
connection = mysql.connector.connect(
    host= os.getenv('DB_HOSTNAME'),       # e.g. 'localhost' or AWS endpoint
    user= os.getenv('DB_USER'),
    password= os.getenv('DB_PASSWORD'),
    database= 'CoinTrades'
)'''

from dotenv import load_dotenv
import mysql.connector
import os

from research.trades_db_utils import create_connection

load_dotenv()
DB_HOSTNAME = os.getenv('DB_HOSTNAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

conn = create_connection(DB_HOSTNAME, 'CoinTrades', DB_USER, DB_PASSWORD)

query = """
SELECT datetime, open, high, low, close, volume, signal
FROM crypto_data
ORDER BY datetime
"""
cursor = conn.cursor()

# Load data
df = pd.read_sql(query, con=conn)
df['datetime'] = pd.to_datetime(df['datetime'])

conn.close()

# Step 2: Clean and sort data
df = df.sort_values('datetime').reset_index(drop=True)
df = df.dropna(subset=['datetime', 'open', 'high', 'low', 'close'])

# Step 2: Create dummy buy/sell signals
if 'signal' not in df.columns or df['signal'].isnull().all():
    df['signal'] = None
    buy_indices = np.random.choice(df.index[10:-10], 5, replace=False)
    sell_indices = np.random.choice(df.index[10:-10], 5, replace=False)
    df.loc[buy_indices, 'signal'] = 'buy'
    df.loc[sell_indices, 'signal'] = 'sell'

/var/folders/ty/5gy6m3k52ps97ms9_x3glvnh0000gn/T/ipykernel_33005/2344153098.py:32: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



DatabaseError: Execution failed on sql '
SELECT datetime, open, high, low, close, volume, signal
FROM crypto_data
ORDER BY datetime
': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'signal
FROM crypto_data
ORDER BY datetime' at line 1

In [ ]:
# Step 3: Plot TradingView-style chart with Plotly
fig = go.Figure()

# Candlestick
fig.add_trace(go.Candlestick(
    x=df['datetime'],
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='Price'
))

# Buy signals
buy_df = df[df['signal'] == 'buy']
fig.add_trace(go.Scatter(
    x=buy_df['datetime'],
    y=buy_df['low'] - 50,
    mode='markers',
    marker=dict(symbol='triangle-up', size=12, color='green'),
    name='Buy Signal'
))

# Sell signals
sell_df = df[df['signal'] == 'sell']
fig.add_trace(go.Scatter(
    x=sell_df['datetime'],
    y=sell_df['high'] + 50,
    mode='markers',
    marker=dict(symbol='triangle-down', size=12, color='red'),
    name='Sell Signal'
))

# Layout settings
fig.update_layout(
    title='Crypto Candlestick Chart with Buy/Sell Signals',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    template='plotly_dark',
    height=700
)

fig.show()